## Import

In [1]:
from ALLCools.clustering import *
from ALLCools.mcds import MCDS
from wmb import brain, aibs, broad, mm10

import pandas as pd
import anndata

import matplotlib.pyplot as plt
from ALLCools.plot import *

import scanpy as sc

In [2]:
n_cell = 5
dataset = 'AIBS_SMART'

## Select cells

In [3]:
cells = pd.read_csv('rna_cells.txt', index_col=0, header=None).index
cells.name = 'cell'
cells.size

100000

## Get adata with basic feature selection

In [4]:
if dataset == 'AIBS_SMART':
    mcds_path = aibs.AIBS_SMART_ZARR_PATH
    gene_map = aibs.get_smart_gene_map()
elif dataset == 'AIBS_TENX':
    mcds_path = aibs.AIBS_TENX_ZARR_PATH
    gene_map = aibs.get_tenx_gene_map()
else:
    mcds_path = broad.BROAD_TENX_ZARR_PATH
    gene_map = broad.get_tenx_gene_map()

In [5]:
rna_ds = MCDS.open(mcds_path, use_obs=cells, var_dim='gene')

In [6]:
rna_ds

<xarray.MCDS>
Dimensions:       (cell: 100000, gene: 45768)
Coordinates:
  * cell          (cell) <U21 'SM-GE4RR_S339_E1-50' ... 'SM-GE5OD_S223_E1-50'
  * gene          (gene) <U28 '0610005C13Rik' '0610006L08Rik' ... 'n-R5s149'
    gene_count    (cell) int64 8070 10279 9587 10946 ... 11637 11640 11620 10677
    read_count    (cell) uint64 626881 573388 586259 ... 451893 595379 661370
Data variables:
    gene_da       (cell, gene) uint32 dask.array<chunksize=(1000, 45768), meta=np.ndarray>
    gene_da_exon  (cell, gene) uint32 dask.array<chunksize=(1000, 45768), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  gene

In [7]:
merfish = anndata.read_h5ad('./adata/merfish_input.h5ad')

In [8]:
#gene_id_not_nan = ~rna_ds.get_index('gene').map(gene_map).isna()
is_merfish_probe = rna_ds.get_index('gene').isin(merfish.var_names)
is_merfish_probe.sum()

483

In [9]:
adata = rna_ds.get_count_adata(da_name='gene_da', use_vars=is_merfish_probe)

Loading chunk 0-10000/100000
Loading chunk 10000-20000/100000
Loading chunk 20000-30000/100000
Loading chunk 30000-40000/100000
Loading chunk 40000-50000/100000
Loading chunk 50000-60000/100000
Loading chunk 60000-70000/100000
Loading chunk 70000-80000/100000
Loading chunk 80000-90000/100000
Loading chunk 90000-100000/100000


In [10]:
adata

AnnData object with n_obs × n_vars = 100000 × 483
    obs: 'count', 'read_count'

In [11]:
# cleanup gene metadata of RNA
assert adata.var_names.duplicated().sum() == 0

# gene_meta = mm10.get_gene_metadata()
# for col in ['chrom', 'start', 'end']:
#     adata.var[col] = gene_meta[col]

## Basic Feature Filtering

In [12]:
# no need to do this, already did in mC

# chroms = [
#     'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
#     'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17',
#     'chr18', 'chr19'
# ]
#
# remove_chromosomes(adata, include_chromosomes=chroms)
# remove_black_list_region(adata, black_list_path=mm10.ENCODE_BLACKLIST_PATH)

In [13]:
if adata.n_obs > n_cell:
    filter_regions(adata, n_cell=n_cell, zscore_abs_cutoff=None)

483 regions remained.


In [14]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata)

In [15]:
adata.write_h5ad('./adata/rna_input.h5ad')

In [16]:
adata

AnnData object with n_obs × n_vars = 100000 × 483
    obs: 'count', 'read_count', 'n_counts'
    var: 'mean', 'std'
    uns: 'log1p'